# Fetching Stock Prices

In [55]:
import pandas as pd
import yfinance as yf
import datetime
from joblib import Parallel, delayed
import multiprocessing
import json 
import json.decoder
import os 
import requests
import pandas_datareader

In [56]:
env_json = {
    'Current_day': 1
}
with open('env.json',"w") as outfile:
    json.dump(env_json,outfile)

In [57]:
tmp_stocks = pd.read_csv('nasdaq.csv')

In [58]:
symbols = list(tmp_stocks['Symbol'])

In [59]:
tmp_json = {}
counter = 0 
sub_count = 0 
for idx, i in enumerate(symbols):
    if "^" in i: 
        continue 
    if sub_count % 500 == 0:
        counter += 1 
        tmp_json[counter] = []
    tmp_json[counter].append(i)
    sub_count += 1 

In [60]:
len(tmp_json)

14

In [61]:
with open("stocks.json","w") as outfile:
    json.dump(tmp_json,outfile)

In [62]:
pandas_datareader.data.DataReader('AAIC' , 'av-intraday', start=datetime.datetime(2022,1,1),end=datetime.datetime(2023,1,14),api_key='QVEGLYRJUK0ABVMT')

,open,high,low,close,volume
2022-12-30 09:31:00,2.9000,2.9100,2.9000,2.9100,255
2022-12-30 09:32:00,2.9100,2.9100,2.9100,2.9100,1000
2022-12-30 09:34:00,2.9101,2.9101,2.9101,2.9101,1000
2022-12-30 09:35:00,2.9200,2.9200,2.9200,2.9200,517
2022-12-30 09:45:00,2.9200,2.9200,2.9200,2.9200,250
...,...,...,...,...,...
2023-01-12 15:56:00,3.0000,3.0000,3.0000,3.0000,1735
2023-01-12 15:57:00,3.0000,3.0000,3.0000,3.0000,643
2023-01-12 15:58:00,3.0000,3.0000,2.9998,2.9998,8280
2023-01-12 15:59:00,3.0100,3.0100,3.0100,3.0100,620
